# HTucker: Hierarchical Tucker Tensor Decomposition

This notebook demonstrates how to use the HTucker package for tensor decomposition and compression.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
import htucker as ht

# Set plotting style
plt.style.use('ggplot')
%matplotlib inline

## 1. Creating a Test Tensor

Let's start by creating a synthetic tensor with some structure to demonstrate compression capabilities.

In [ ]:
# Create a tensor with low-rank structure plus noise
def create_test_tensor(shape=(10, 8, 6, 4), rank=2, noise_level=0.05):
    """Create a tensor with specified shape, approximate rank, and noise level.
    
    Args:
        shape: Shape of the tensor
        rank: Approximate rank for each mode
        noise_level: Level of noise to add
        
    Returns:
        ndarray: The created tensor
    """
    # Create factors for each dimension
    factors = []
    for dim_size in shape:
        # Create random factors
        factor = np.random.rand(dim_size, rank)
        factors.append(factor)
    
    # Create core tensor
    core = np.random.rand(*([rank] * len(shape)))
    
    # Initialize tensor
    tensor = np.zeros(shape)
    
    # Build tensor from factors - this creates a low-rank structure
    # This is a simple implementation and not the most efficient way
    # For high-dimensional tensors, one would use proper tensor operations
    index_ranges = [range(s) for s in shape]
    for idx in np.ndindex(*shape):
        value = 0
        for r_idx in np.ndindex(*([rank] * len(shape))):
            tmp = core[r_idx]
            for i, (dim_idx, r) in enumerate(zip(idx, r_idx)):
                tmp *= factors[i][dim_idx, r]
            value += tmp
        tensor[idx] = value
    
    # Add noise
    if noise_level > 0:
        noise = np.random.normal(0, noise_level * np.std(tensor), size=shape)
        tensor += noise
    
    return tensor

# Create tensor
np.random.seed(42)  # For reproducibility
tensor_shape = (10, 8, 6, 4)
tensor = create_test_tensor(shape=tensor_shape, rank=3, noise_level=0.1)

# Print information
print(f"Tensor shape: {tensor.shape}")
print(f"Tensor size: {tensor.size} elements")
print(f"Memory usage: {tensor.nbytes / 1024:.2f} KB")

# Show a slice of the tensor
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.imshow(tensor[0, :, :, 0], cmap='viridis')
plt.title('Tensor Slice [0, :, :, 0]')
plt.colorbar()

plt.subplot(1, 2, 2)
plt.imshow(tensor[:, :, 0, 0], cmap='viridis')
plt.title('Tensor Slice [:, :, 0, 0]')
plt.colorbar()
plt.tight_layout()

## 2. Creating a Dimension Tree

The HTucker decomposition requires a dimension tree that defines how the tensor dimensions are hierarchically organized.

In [ ]:
# Create dimension tree
dim_tree = ht.createDimensionTree(tensor, 2, 1)

# Print dimension tree information
print(f"Dimension tree created with {dim_tree._leafCount} leaves")
print("\nDimension tree structure:")
for node in dim_tree.nodes:
    if node.parent is not None:
        parent_dims = node.parent.dimensions
    else:
        parent_dims = "None (root)"
    print(f"Node dimensions: {node.dimensions}, Parent: {parent_dims}")

# Visualize the dimension tree structure (simple text representation)
def print_tree(node, level=0):
    indent = "  " * level
    print(f"{indent}Node: {node.dimensions}")
    for child in node.children:
        print_tree(child, level + 1)

print("\nTree structure:")
print_tree(dim_tree.root)

## 3. Root-to-Leaf Compression

Now let's compress the tensor using the HTucker root-to-leaf approach, which applies HOSVD starting from the root node.

In [ ]:
# Initialize HTucker object
htd_r2l = ht.HTucker()
htd_r2l.initialize(tensor)

# Compress using root-to-leaf approach
start_time = time.time()
htd_r2l.compress_root2leaf(tensor)
r2l_time = time.time() - start_time
print(f"Root-to-leaf compression time: {r2l_time:.4f} seconds")

# Check if compression succeeded
print(f"Compression completed: {htd_r2l._iscompressed}")

# Reconstruct the tensor
start_time = time.time()
reconstructed_r2l = htd_r2l.reconstruct_all()
reconstruct_time = time.time() - start_time
print(f"Reconstruction time: {reconstruct_time:.4f} seconds")

# Calculate error
rel_error_r2l = np.linalg.norm(reconstructed_r2l - tensor) / np.linalg.norm(tensor)
print(f"Relative reconstruction error: {rel_error_r2l:.8e}")

# Memory usage
memory_r2l = htd_r2l.get_memory_size()
compression_ratio_r2l = tensor.nbytes / memory_r2l
print(f"Memory usage: {memory_r2l / 1024:.2f} KB")
print(f"Compression ratio: {compression_ratio_r2l:.2f}x")

## 4. Leaf-to-Root Compression

Now let's try the leaf-to-root compression approach, which applies truncated SVD starting from the leaf nodes.

In [ ]:
# Initialize HTucker object
htd_l2r = ht.HTucker()
htd_l2r.initialize(tensor, dimension_tree=dim_tree)

# Set relative tolerance for SVD truncation
htd_l2r.rtol = 1e-6
print(f"Using relative tolerance: {htd_l2r.rtol}")

# Compress using leaf-to-root approach
start_time = time.time()
htd_l2r.compress_leaf2root(tensor, dim_tree)
l2r_time = time.time() - start_time
print(f"Leaf-to-root compression time: {l2r_time:.4f} seconds")

# Reconstruct the tensor
start_time = time.time()
reconstructed_l2r = htd_l2r.reconstruct_all()
reconstruct_time_l2r = time.time() - start_time
print(f"Reconstruction time: {reconstruct_time_l2r:.4f} seconds")

# Calculate error
rel_error_l2r = np.linalg.norm(reconstructed_l2r - tensor) / np.linalg.norm(tensor)
print(f"Relative reconstruction error: {rel_error_l2r:.8e}")

# Memory usage
memory_l2r = htd_l2r.get_memory_size()
compression_ratio_l2r = tensor.nbytes / memory_l2r
print(f"Memory usage: {memory_l2r / 1024:.2f} KB")
print(f"Compression ratio: {compression_ratio_l2r:.2f}x")

# Show ranks
print(f"\nRanks after compression: {htd_l2r.get_ranks()}")

## 5. Comparison of Compression Methods

Let's compare the results of the two compression methods.

In [ ]:
# Comparison table
comparison = {
    'Method': ['Root-to-Leaf', 'Leaf-to-Root'],
    'Compression Time (s)': [r2l_time, l2r_time],
    'Reconstruction Time (s)': [reconstruct_time, reconstruct_time_l2r],
    'Relative Error': [rel_error_r2l, rel_error_l2r],
    'Memory (KB)': [memory_r2l / 1024, memory_l2r / 1024],
    'Compression Ratio': [compression_ratio_r2l, compression_ratio_l2r]
}

# Print comparison
from tabulate import tabulate
try:
    from tabulate import tabulate
    headers = comparison.keys()
    rows = list(zip(*comparison.values()))
    print(tabulate(rows, headers=headers, floatfmt='.4f'))
except ImportError:
    print("Install tabulate for better formatting: pip install tabulate")
    for key, values in comparison.items():
        print(f"{key}:")
        for method, value in zip(comparison['Method'], values):
            print(f"  {method}: {value}")

# Visualization of original vs reconstructed
slice_idx = (0, slice(None), slice(None), 0)

plt.figure(figsize=(15, 4))
plt.subplot(1, 3, 1)
plt.imshow(tensor[slice_idx], cmap='viridis')
plt.title('Original Tensor')
plt.colorbar()

plt.subplot(1, 3, 2)
plt.imshow(reconstructed_r2l[slice_idx], cmap='viridis')
plt.title('Root-to-Leaf Reconstruction')
plt.colorbar()

plt.subplot(1, 3, 3)
plt.imshow(reconstructed_l2r[slice_idx], cmap='viridis')
plt.title('Leaf-to-Root Reconstruction')
plt.colorbar()

plt.tight_layout()

## 6. Effect of Tolerance Parameter

Let's explore how the tolerance parameter affects compression ratio and accuracy.

In [ ]:
# Test different tolerance values
tolerances = [1e-10, 1e-8, 1e-6, 1e-4, 1e-2, 1e-1]
results = []

for tol in tolerances:
    # Initialize HTucker object
    htd_test = ht.HTucker()
    htd_test.initialize(tensor, dimension_tree=dim_tree)
    htd_test.rtol = tol
    
    # Compress
    htd_test.compress_leaf2root(tensor, dim_tree)
    
    # Reconstruct
    reconstructed = htd_test.reconstruct_all()
    
    # Calculate error and memory
    rel_error = np.linalg.norm(reconstructed - tensor) / np.linalg.norm(tensor)
    memory = htd_test.get_memory_size()
    compression_ratio = tensor.nbytes / memory
    
    results.append({
        'tolerance': tol,
        'rel_error': rel_error,
        'compression_ratio': compression_ratio,
        'memory_kb': memory / 1024,
        'ranks': htd_test.get_ranks()
    })
    
    print(f"Tolerance: {tol:.1e}, Error: {rel_error:.8e}, Compression: {compression_ratio:.2f}x")

# Plot results
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.loglog([r['tolerance'] for r in results], [r['rel_error'] for r in results], 'o-')
plt.xlabel('Tolerance')
plt.ylabel('Relative Error')
plt.title('Error vs Tolerance')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.semilogx([r['tolerance'] for r in results], [r['compression_ratio'] for r in results], 'o-')
plt.xlabel('Tolerance')
plt.ylabel('Compression Ratio')
plt.title('Compression Ratio vs Tolerance')
plt.grid(True)

plt.tight_layout()

## 7. Incremental Update

One of the key features of HTucker is the ability to incrementally update the decomposition with new data.

In [ ]:
# Create a new tensor with slightly different structure
new_tensor = create_test_tensor(shape=tensor_shape, rank=3, noise_level=0.15)

# Use our existing decomposition and update it
print("Original ranks before update:", htd_l2r.get_ranks())

# Perform incremental update
start_time = time.time()
htd_l2r.incremental_update(new_tensor)
update_time = time.time() - start_time
print(f"Incremental update time: {update_time:.4f} seconds")
print("Ranks after update:", htd_l2r.get_ranks())

# Project and reconstruct the new tensor
projected = htd_l2r.project(new_tensor)
reconstructed_new = htd_l2r.reconstruct(projected)

# Calculate error
rel_error_new = np.linalg.norm(reconstructed_new - new_tensor) / np.linalg.norm(new_tensor)
print(f"Relative reconstruction error for new tensor: {rel_error_new:.8e}")

# Compare with a fresh decomposition
htd_fresh = ht.HTucker()
htd_fresh.initialize(new_tensor, dimension_tree=dim_tree)
htd_fresh.rtol = 1e-6

start_time = time.time()
htd_fresh.compress_leaf2root(new_tensor, dim_tree)
fresh_time = time.time() - start_time
print(f"Fresh compression time: {fresh_time:.4f} seconds")
print(f"Speed-up from incremental update: {fresh_time/update_time:.2f}x")

reconstructed_fresh = htd_fresh.reconstruct_all()
rel_error_fresh = np.linalg.norm(reconstructed_fresh - new_tensor) / np.linalg.norm(new_tensor)
print(f"Relative reconstruction error for fresh decomposition: {rel_error_fresh:.8e}")

# Visualization
plt.figure(figsize=(15, 4))

plt.subplot(1, 3, 1)
plt.imshow(new_tensor[slice_idx], cmap='viridis')
plt.title('New Tensor')
plt.colorbar()

plt.subplot(1, 3, 2)
plt.imshow(reconstructed_new[slice_idx], cmap='viridis')
plt.title(f'Incremental Update\nError: {rel_error_new:.2e}')
plt.colorbar()

plt.subplot(1, 3, 3)
plt.imshow(reconstructed_fresh[slice_idx], cmap='viridis')
plt.title(f'Fresh Decomposition\nError: {rel_error_fresh:.2e}')
plt.colorbar()

plt.tight_layout()

## 8. Batch Processing

HTucker can efficiently handle batch data by treating one dimension as a batch dimension.

In [ ]:
# Create a batch tensor
batch_size = 5
batch_tensor = np.zeros((*tensor_shape[:-1], batch_size))

# Fill with different test tensors
for b in range(batch_size):
    # Create tensor with varying parameters
    noise_level = 0.05 + 0.05 * b  # Increasing noise level
    batch_tensor[..., b] = create_test_tensor(
        shape=tensor_shape[:-1], 
        rank=3, 
        noise_level=noise_level
    )

print(f"Batch tensor shape: {batch_tensor.shape}")

# Create dimension tree (excluding batch dimension)
batch_tree = ht.createDimensionTree(batch_tensor.shape[:-1], 2, 1)

# Initialize HTucker for batch processing
batch_dimension = len(batch_tensor.shape) - 1  # Last dimension is batch
htd_batch = ht.HTucker()
htd_batch.initialize(batch_tensor, dimension_tree=batch_tree, 
                      batch=True, batch_dimension=batch_dimension)
htd_batch.rtol = 1e-6

# Compress batch tensor
start_time = time.time()
htd_batch.compress_leaf2root_batch(batch_tensor, batch_tree, batch_dimension=batch_dimension)
batch_time = time.time() - start_time
print(f"Batch compression time: {batch_time:.4f} seconds")

# Check if batch count matches
print(f"Batch count in HTucker: {htd_batch.batch_count}")
print(f"Expected batch count: {batch_size}")

# Reconstruct individual batch samples
batch_errors = []
reconstructions = []

for b in range(batch_size):
    # Extract sample
    sample = batch_tensor[..., b:b+1]
    
    # Project
    projected = htd_batch.project(sample, batch=True, batch_dimension=batch_dimension)
    
    # Reconstruct
    reconstructed = htd_batch.reconstruct(projected, batch=True, batch_dimension=batch_dimension)
    reconstructed = reconstructed[..., 0]  # Remove singleton batch dimension
    
    # Calculate error
    original = sample[..., 0]  # Remove singleton batch dimension
    error = np.linalg.norm(reconstructed - original) / np.linalg.norm(original)
    batch_errors.append(error)
    reconstructions.append(reconstructed)
    
    print(f"Batch {b} - Relative error: {error:.8e}")

# Plot batch results
plt.figure(figsize=(12, 8))

for b in range(batch_size):
    # Original
    plt.subplot(batch_size, 3, 3*b + 1)
    plt.imshow(batch_tensor[0, :, :, b], cmap='viridis')
    if b == 0:
        plt.title('Original')
    plt.ylabel(f"Batch {b}")
    plt.colorbar()
    
    # Reconstructed
    plt.subplot(batch_size, 3, 3*b + 2)
    plt.imshow(reconstructions[b][0, :, :], cmap='viridis')
    if b == 0:
        plt.title('Reconstructed')
    plt.colorbar()
    
    # Error
    plt.subplot(batch_size, 3, 3*b + 3)
    diff = batch_tensor[0, :, :, b] - reconstructions[b][0, :, :]
    plt.imshow(diff, cmap='RdBu_r')
    if b == 0:
        plt.title('Difference')
    plt.colorbar()

plt.tight_layout()

## 9. Conclusion

In this notebook, we've demonstrated the key features of the HTucker package:

1. Root-to-leaf compression
2. Leaf-to-root compression with error control
3. Incremental updates for efficient processing of new data
4. Batch processing for handling multiple tensors simultaneously

HTucker is particularly useful for high-dimensional tensors where traditional tensor decomposition methods become computationally infeasible. By exploiting the hierarchical structure, HTucker achieves efficient compression and reconstruction while maintaining accuracy.